<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/period-and-revolution-time-rigid-pendulum-scf196/blob/main/period_and_revolution_time_rigid_pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Psuedocode

1. **Import packages:** `numpy` for special functions and linear algebra, `matplotlib.pyplot` for plotting, `from scipy.integrate import solve_ivp, quad` for solving the differential equation & integration.
2. **Define ODE function & integrand:** keeping in mind that when $\dot{\theta}_0 \ge 2\omega_0$, integrate in $\theta$ from $0$ to $2\pi$: $$T = \frac {1} {\sqrt{2 \omega_0 ^2}} \int_{0}^{2 \pi} \frac{d \theta}{\sqrt{\frac{\dot{\theta_0 ^2}}{2 \omega_0 ^2}}-1+\cos{\theta}}$$
and when $\dot{\theta}_0 < 2\omega_0$, find $\theta_{peak}$ and use quarter-period approach, integrating $\theta$ from $0$ to $\theta_{peak}$:
$$T = \frac {4} {\sqrt{2 \omega_0 ^2}} \int_{0}^{\theta_{peak}} \frac{d \theta}{\sqrt{\frac{\dot{\theta_0 ^2}}{2 \omega_0 ^2}}-1+\cos{\theta}}$$
3. **Define utilities:** `basic_plot(x, y, xlabel=None, ylabel=None, figsize=(3, 3))` for plotting, `ang_vel_zero_event(t, y, omega_0=1)` for events (to find difference from peak to peak)
4. **Extract the period using the differential equation:** call `quad` after defining the coefficient & integrand
5. **Extract the period using the differential equation:** use `solve_ivp` and `sol.t_events` after solving to find the peak to peak distances

In [149]:
# Import packages

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, quad

In [150]:
# Define ODE function & integrand
# two cases: revolutionary motion where ang_vel > omega_0 and oscillatory motion where ang_vel < omega_0
# revolutionary motion shows a full change by 2π when one rotation is made
# oscillatory motion shows the period when we return to theta_0
# define two different integrands used for the different cases

# rigid pendulum ode
def rigid_pendulum_ode(t, y, omega_0=1):
  """
  The ordinary differential equation for a rigid pendulum.
  """
  theta, ang_vel = y # unpacking state variable
  dtheta_dt = ang_vel # first deriv
  d2theta_dt2 = -omega_0**2 * np.sin(theta) # second deriv
  dy_dt = [dtheta_dt, d2theta_dt2] # deriv of state variable
  return dy_dt

# integrand for full revolution, for quad
def period_integrand_rev(theta, ang_vel_0, omega_0=1):
  """
  The integrand for finding the revolution time of a pendulum in revolving motion.
  """
  integrand_rev = 1 / np.sqrt(ang_vel_0**2 - 2 * omega_0**2 * (1 - np.cos(theta)))
  return integrand_rev

# integrand for oscillatory motion, for quad
def period_integrand_osc(theta, theta_0, omega_0=1):
  """
  The integrand for finding the period time of a pendulum in oscillatory motion.
  """
  integrand_osc = (4/(np.sqrt(2)*omega_0)) / (np.sqrt(np.cos(theta)-np.cos(theta_0)))
  return integrand_osc

In [151]:
# Define utilities

# basic plotting

def basic_plot(x, y, xlabel=None, ylabel=None, figsize=(3, 3)):
  """
  stuff
  """
  plt.figure(figsize=figsize)
  plt.plot(x, y)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  return plt.show()

# define event function: theta = 0
# event for when theta returns to 0 in case where theta_0 = 0
def theta_zero_event(t, y, omega_0=1):
  """
  Event that becomes true when ang_vel = 0.
  """
  theta, ang_vel = y # unpacking state variable
  return theta # set to find values where theta = 0
# add required attributes to event function
theta_zero_event.terminal = False # keep integrating after events are found

# define event function: ang_vel = 0
def ang_vel_zero_event(t, y, omega_0=1):
  """
  Event that becomes true when ang_vel = 0.
  """
  theta, ang_vel = y # unpacking state variable
  return ang_vel # set to find values where ang_vel = 0
# add required attributes to event function
ang_vel_zero_event.terminal = False # keep integrating after events are found
ang_vel_zero_event.direction = -1 # from positive to negative

In [152]:
# quad

ang_vel_0 = 3 # initial angular velocity
theta_0 = np.pi/2 # initial angle
omega_0 = 1 # natural frequency

# first quad is for revolutionary motion, second is for oscillatory
rev_quad, err = quad(period_integrand_rev, 0, 2*np.pi, args=(ang_vel_0, omega_0)) # quad gives two arguments, period & error
period_quad, err = quad(period_integrand_osc, 0, np.pi/2, args=(theta_0, omega_0)) # quad gives two arguments, period & error

print(rev_quad)
print(period_quad)

2.412889993982118
7.416298709205221


In [153]:
# solve_ivp

t_min = 0
t_max = 100
t_span = [t_min, t_max] # time span for solve_ivp

# state variable
ang_vel_0 = 3 # initial angular velocity
omega_0 = 1 # natural frequency
theta_0 = np.pi/2 # initial angle
y_0_rev = [0, ang_vel_0] # initial condition for revolutionary motion
y_0_period = [theta_0, 0] # initial condition for oscillatory motion

# solve differential equation

sol_rev = solve_ivp(rigid_pendulum_ode,
                    t_span,
                    y_0_rev,
                    args=(ang_vel_0,),
                    max_step=0.01,
                    dense_output=True,
                    events=ang_vel_zero_event
                    )

sol_period = solve_ivp(rigid_pendulum_ode,
                       t_span,
                       y_0_period,
                       args=(theta_0,),
                       max_step=0.01,
                       dense_output=True,
                       events=ang_vel_zero_event
                       )

rev_sol = np.mean(np.diff(sol_rev.t_events))
print(rev_sol)

period_sol = np.mean(np.diff(sol_period.t_events))
print(period_sol)

2.2476671395306096
4.721362396046851


In [154]:
# percent difference

percent_diff_rev = abs(rev_sol - rev_quad) / rev_sol * 100
print(percent_diff_rev)

percent_diff_period = abs(period_quad - period_sol) / period_quad * 100
print(percent_diff_period)

# percent difference is very high and very unreasonable
# one method must be wrong, unsure how to code the problem properly or plot

7.350859544354615
36.338022763476
